In [20]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pickle
from glob import glob
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# RNN

In [21]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        
        return out

# Dataset

In [22]:
class ArgoverseDataset(torch.utils.data.Dataset):
    """Dataset class for Argoverse"""
    
    def __init__(self, 
                 data_path,
                 sample_indices):
        super(ArgoverseDataset, self).__init__()
        
        self.data_path = data_path
        self.sample_indices = sample_indices
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.sample_indices)

    def __getitem__(self, idx):
        
        # Load one scene
        pkl_path = self.pkl_list[self.sample_indices[idx]]
        with open(pkl_path, 'rb') as f:
            scene = pickle.load(f)
            
        # the index of agent to be predicted 
        pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
        
        # input: p_in & v_in; output: p_out
        p_in_raw = scene['p_in'][pred_id]
        p_out_raw = scene['p_out'][pred_id]
        v_in_raw = scene['v_in'][pred_id]
        v_out_raw = scene['v_out'][pred_id]
        lane_scene = scene['lane']
        
        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_normalized = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        p_out_normalized = (p_out_raw - min_vecs)/(max_vecs - min_vecs)
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_normalized = v_in_raw / v_in_norm
        v_out_norm = np.linalg.norm(v_out_raw, axis=1, keepdims=True)
        v_out_norm = np.where(v_out_norm == 0.0, 1.0, v_out_norm)
        v_out_normalized = v_out_raw / v_out_norm
        inp = np.concatenate((p_in_normalized,v_in_normalized),axis=1)
        output = np.concatenate((p_out_normalized, v_out_normalized), axis=1)
        full = np.vstack((inp, output))

        stack = []
        for i in range(output.shape[0]):
            stack.append(full[i:i+inp.shape[0], :])
        stack = np.stack(stack)
        
        # Convert to float torch tensor
        return torch.from_numpy(stack).float(), torch.from_numpy(output).float()

In [23]:
class PVDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 x_data_path,
                 y_data_path,
                 sample_indices):
        super(PVDataset, self).__init__()
        
        self.X = np.load(x_data_path)
        self.y = np.load(y_data_path)
        self.sample_indices = sample_indices

    def __len__(self):
        return len(self.sample_indices)
    
    def __getitem__(self, idx):
        sample_x = self.X[idx]
        sample_y = self.y[idx]
        return torch.from_numpy(sample_x).float(), torch.from_numpy(sample_y).float()


# Hyper-parameter

In [33]:
batch_size = 512
in_dim = 4
in_len = 19
out_dim = 4
out_len = 30
hidden_dim = 128
num_layers = 2
learning_rate = 0.01
decay_rate = 0.95
num_epoch = 1000

# Data Loader

In [25]:
train_path = "./train"

# total number of scenes
indices = np.arange(0, 205942)

# train-valid split
np.random.shuffle(indices)
train_indices = indices[:180000]
valid_indices = indices[180000:]

# define datasets
train_set = PVDataset('train_X.npy', 'train_y.npy', train_indices)
valid_set = PVDataset('train_X.npy', 'train_y.npy', valid_indices)

# create dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

# Model, Loss Function and Optimizer

In [26]:
model = LSTMModel(input_size=in_dim, hidden_size=hidden_dim, num_layers=num_layers, output_size=out_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=decay_rate)

# Training

In [27]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def train_epoch(train_loader, model, optimizer, criterion):
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader):
        # inputs = torch.flatten(inputs, end_dim=1)
        inputs = inputs.to(device)
        # targets = torch.flatten(targets, end_dim=1)
        targets = targets.to(device)

        optimizer.zero_grad()

        # outputs = torch.zeros(targets.shape)
        # for i in range(outputs.size(1)-in_len):
        #     input_tensor = inputs[:, i:i+in_len, :]
        #     input_tensor = input_tensor.to(device)
        #     output_tensor = model(input_tensor)
        #     outputs[:, i, :] = output_tensor
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    return running_loss / len(train_loader)

def eval_epoch(valid_loader, model, criterion):
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in valid_loader:
            # inputs = torch.flatten(inputs, end_dim=1)
            inputs = inputs.to(device)
            # targets = torch.flatten(targets, end_dim=1)
            targets = targets.to(device)

            # outputs = torch.zeros(targets.shape)
            # for i in range(outputs.size(1)-in_len):
            #     input_tensor = inputs[:, i:i+in_len, :]
            #     input_tensor = input_tensor.to(device)
            #     output_tensor = model(input_tensor)
            #     outputs[:, i, :] = output_tensor
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)

    return val_loss / len(valid_loader.dataset)

In [28]:
train_rmse = []
valid_rmse = []
min_rmse = 10e8

for epoch in range(num_epoch):
    start = time.time()

    model.train() # if you use dropout or batchnorm. 
    train_rmse.append(train_epoch(train_loader, model, optimizer, criterion))

    model.eval()
    valid_rmse.append(eval_epoch(valid_loader, model, criterion))

    # save the best model
    if valid_rmse[-1] < min_rmse:
        min_rmse = valid_rmse[-1] 
        best_model = model
        torch.save([best_model, epoch, get_lr(optimizer)], "model.pth")

    end = time.time()
    
    # Early Stopping
    if (len(train_rmse) > 100 and np.mean(valid_rmse[-5:]) >= np.mean(valid_rmse[-10:-5])):
            break       

    # Learning Rate Decay        
    scheduler.step()
    
    print("Epoch {} | T: {:0.2f} | Train RMSE: {:0.5f} | Valid RMSE: {:0.5f}".format(epoch + 1, (end-start) / 60, train_rmse[-1], valid_rmse[-1]))

  0%|          | 0/352 [00:00<?, ?it/s]

Epoch 1 | T: 1.57 | Train RMSE: 21.01097 | Valid RMSE: 0.02720


  0%|          | 0/352 [00:00<?, ?it/s]

Epoch 2 | T: 1.62 | Train RMSE: 14.41015 | Valid RMSE: 0.02699


  0%|          | 0/352 [00:00<?, ?it/s]

...

Epoch 44 | T: 1.55 | Train RMSE: 9.76805 | Valid RMSE: 0.01743


  0%|          | 0/352 [00:00<?, ?it/s]

Epoch 45 | T: 1.55 | Train RMSE: 9.67842 | Valid RMSE: 0.01719


 29%|██▉       | 103/352 [00:26<01:03,  3.93it/s]


KeyboardInterrupt: 

In [32]:
best_model, _, _ = torch.load('model.pth')
best_model

LSTMModel(
  (lstm): LSTM(4, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=4, bias=True)
)

# Evaluation and Submission

In [54]:
test_path = "./test"
test_pkl_list = glob(os.path.join(test_path, '*'))
test_pkl_list.sort()

test_preds = []
best_model.eval()
for test_pkl_path in tqdm(test_pkl_list):
    with open(test_pkl_path, 'rb') as f:
        scene = pickle.load(f)
        pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
        # input: p_in & v_in; output: p_out
        p_in_raw = scene['p_in'][pred_id]
        v_in_raw = scene['v_in'][pred_id]
        lane_scene = scene['lane']
        
        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_normalized = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_normalized = v_in_raw / v_in_norm
        inp = np.concatenate((p_in_normalized,v_in_normalized),axis=1)
        inp = torch.from_numpy(inp).float().unsqueeze(0).to(device)

        output = []
        for i in range(out_len):
            test_x = inp[:,-in_len:,:]
            pred = best_model(test_x)
            output.append(pred)
            inp = torch.cat((inp, pred.unsqueeze(0)), dim=1)
        output = torch.vstack(output)
        output = output.detach().numpy()[:,:2]
        
        # De-Normalization ! 
        output = output * (max_vecs[:2] - min_vecs[:2]) +  min_vecs[:2]
        test_preds.append(output)

100%|██████████| 3200/3200 [05:35<00:00,  9.53it/s]


# Generate Submission File

In [55]:
# # Submission Files
sample_sub = pd.read_csv('sample_submission.csv')

In [59]:
# Convert to int
predictions = np.concatenate(test_preds).reshape(len(test_preds), -1).astype(int)
sub_df = pd.DataFrame(np.c_[sample_sub["ID"], predictions], columns=[np.r_[["ID"], ["v" + str(i) for i in range(1, 61)]]])
sub_df.to_csv('rnn_test_submission.csv', index=None)

In [58]:
sub_df.dtypes

ID     float64
v1     float64
v2     float64
v3     float64
v4     float64
        ...   
v56    float64
v57    float64
v58    float64
v59    float64
v60    float64
Length: 61, dtype: object